# EO4SD SHORELINE CHANGE MAPPING AND FORECASTING

This code has been modifed by Carpenter (2020) for the project Earth Observation for Sustainable Development. Below demonstrates an example download for Benin and Togo's Coastline between 2000-2020.

This software is based on scripts and code developed by:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 20+ years at their site of interest.
There are three main steps:
1. Retrieval of median composite satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution

## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [2]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects

## 1. Retrieval of the images from GEE

To retrieve from the GEE server the available satellite images cropped around the user-defined region of coastline for the particular time period of interest, the following variables are required:

* Coordinate list: a list of the coordinates of the region of interest (longitude/latitude pairs in WGS84) – see section 'create coordinate list in User handbook to extract ROI from study site
* all_dates: dates over which the images will be retrieved (e.g., dates = ['2017-12-01', '2018-01-01'])
* all_sats: satellite missions to consider (e.g., sat_list = ['L7', 'L8', 'S2'] for Landsat 7, 8 and Sentinel-2 collections)
* sitename: name of the site (this is the name of the subfolder where the images and other accompanying files will be stored)
* filepath: filepath to the directory where the data will be stored

Make sure the area of your ROI is smaller than 100 km2 (if larger split it into smaller ROIs) - GEE limits download size

The function `SDS_download.check_images_available(inputs)` will print the number of images used in the median composite. 

In [ ]:
coordinate_list =([[1.29690864400004,6.15137585000002],[1.29690864400004,6.10009291000006],[1.19790624200004,6.15137585000002],[1.19790624200004,6.10009291000006]]),\
([[1.38873032500004,6.18742338100003],[1.38873032500004,6.13901283000007],[1.28920399000003,6.18742338100003],[1.28920399000003,6.13901283000007]]),\
([[1.48398541600005,6.21198217900007],[1.48398541600005,6.16972946500005],[1.38371324500002,6.21198217900007],[1.38371324500002,6.16972946500005]]),\
([[1.58042952600005,6.23227336700006],[1.58042952600005,6.19343642700005],[1.47995056800005,6.23227336700006],[1.47995056800005,6.19343642700005]]),\
([[1.67696272400002,6.25348088100003],[1.67696272400002,6.21531279800007],[1.57645968700007,6.25348088100003],[1.57645968700007,6.21531279800007]]),\
([[1.77364700400005,6.27389994300006],[1.77364700400005,6.23653869900005],[1.67312191700006,6.27389994300006],[1.67312191700006,6.23653869900005]]),\
([[1.86928619400004,6.30005936100002],[1.86928619400004,6.26096190900006],[1.76881804700002,6.30005936100002],[1.76881804700002,6.26096190900006]]),\
([[1.95899271500002,6.31399944800006],[1.95899271500002,6.28289237200005],[1.86508446300007,6.31399944800006],[1.86508446300007,6.28289237200005]]),\
([[2.05649633200005,6.33071125400005],[2.05649633200005,6.29627151800003],[1.95595420500007,6.33071125400005],[1.95595420500007,6.29627151800003]]),\
([[2.15477967200007,6.34174422100006],[2.15477967200007,6.30827463300005],[2.05425324300006,6.34174422100006],[2.05425324300006,6.30827463300005]]),\
([[2.25257140100007,6.35265504900002],[2.25257140100007,6.32340083500003],[2.15223313700005,6.35265504900002],[2.15223313700005,6.32340083500003]]),\
([[2.35028169600002,6.35909331300007],[2.35028169600002,6.33398027500004],[2.25031238200006,6.35909331300007],[2.25031238200006,6.33398027500004]]),\
([[2.44952715900007,6.35850084700007],[2.44952715900007,6.33184616500006],[2.34881337200005,6.35850084700007],[2.34881337200005,6.33184616500006]]),\
([[2.54681978300005,6.37081751400007],[2.54681978300005,6.34725958000007],[2.44703476400002,6.37081751400007],[2.44703476400002,6.34725958000007]]),\
([[2.64569441800006,6.37483411400007],[2.64569441800006,6.35060457600002],[2.54582678500003,6.37483411400007],[2.54582678500003,6.35060457600002]]),\
([[2.74424146600006,6.38839034600005],[2.74424146600006,6.35851337500003],[2.64386332000004,6.38839034600005],[2.64386332000004,6.35851337500003]])

counter = 0

for polygon in coordinate_list:

# region of interest (longitude, latitude)
    polygon = coordinate_list[counter]

    all_dates = ([['2000-01-01', '2000-12-31'],['2001-01-01', '2001-12-31'],['2002-01-01', '2002-12-31'],['2003-01-01', '2003-12-31'],\
                 ['2004-01-01', '2004-12-31'],['2005-01-01', '2005-12-31'],['2006-01-01', '2006-12-31'],['2007-01-01', '2007-12-31'],\
                 ['2008-01-01', '2008-12-31'],['2009-01-01', '2009-12-31'],['2010-01-01', '2010-12-31'],['2011-01-01', '2011-12-31'],\
                 ['2013-01-01', '2013-12-31'],['2014-01-01', '2014-12-31'],['2015-01-01', '2015-12-31'],\
                 ['2016-01-01', '2016-12-31'],['2017-01-01', '2017-12-31'],['2018-01-01', '2018-12-31'],['2019-01-01', '2019-12-31'],\
                 ['2020-01-01', '2020-06-30']])

    all_sats = ([['L7'],['L7'],['L7'],['L7'],\
                 ['L7'],['L7'],['L7'],['L7'],\
                 ['L7'],['L7'],['L7'],['L7'],\
                 ['L8'],['L8'],['L8'],\
                 ['S2'],['S2'],['S2'],['S2'],\
                 ['S2']])
    
    rolling = 0

    for dates in all_dates:
        # date range
        dates = all_dates[rolling]
        # satellite missions
        sat_list = all_sats[rolling]
        # name of the site
        foldernumber = counter
        sitename = 'Benin_Togo_' + str(foldernumber) 
        # directory where the data will be stored
        filepath = os.path.join(os.getcwd(), 'data')
        # put all the inputs into a dictionnary
        inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath': filepath, 'rolling': str(rolling)}

        # before downloading the images, check how many images are available for your inputs
        SDS_download.check_images_available(inputs);
        
        settings = { 
            # general parameters:
            'cloud_thresh': 0.3,        # threshold on maximum cloud cover
            'output_epsg': 32631,        # epsg code of spatial reference system desired for the output   
            # quality control:
            #'check_detection': False,    # if True, shows each shoreline detection to the user for validation
            #'save_figure': False,        # if True, saves a figure showing the mapped shoreline for each image
            # add the inputs defined previously
            'inputs': inputs,
            # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
            'min_beach_area': 1000,     # minimum area (in metres^2) for an object to be labelled as a beach
            'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
            'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
            'cloud_mask_issue': True,  # switch this parameter to True if sand pixels are masked (in black) on many images  
            'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
        }

        inputs['include_T2'] = False
        metadata = SDS_download.retrieve_images(inputs,settings)

        metadata = SDS_download.get_metadata(inputs)
          
        rolling = rolling + 1
##Batch shoreline detection
    %matplotlib qt
    output = SDS_shoreline.extract_shorelines(metadata, settings)

    counter = counter + 1

Images available between 2000-01-01 and 2000-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L7: 3 images
  Total: 3 images
Images available between 2000-01-01 and 2000-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L7: 3 images
  Total: 3 images

Median Processed
Displacement Calculated
Registered
Downloaded

Images available between 2001-01-01 and 2001-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L7: 5 images
  Total: 5 images
Images available between 2001-01-01 and 2001-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L7: 5 images
  Total: 5 images

Median Processed
Displacement Calculated
Registered
